## Merging injury and gamelog data

In [2]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from sklearn.preprocessing import OneHotEncoder

/Users/freedom/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [218]:
# Import files
injury_file = 'injuries.csv'
injury = pd.read_csv(injury_file)

game_file = 'gamelogs.csv'
games = pd.read_csv(game_file)

In [219]:
injury.head()

,Date,DL_length,player_id
0,20000323,60,thomj005
1,20000401,15,alvaw001
2,20000401,15,osuna002
3,20000408,15,hamid001
4,20000408,15,timlm001


In [220]:
injury.shape

(9288, 3)

In [221]:
games.head()

,date,num_game,day,v_team,v_league,v_team_game_num,h_team,h_league,h_team_game_num,v_team_score,...,v_pl_9_id,h_pl_1_id,h_pl_2_id,h_pl_3_id,h_pl_4_id,h_pl_5_id,h_pl_6_id,h_pl_7_id,h_pl_8_id,h_pl_9_id
0,20000329,0,Wed,CHN,NL,1,NYN,NL,1,5,...,liebj001,hendr001,hamid001,alfoe001,piazm001,ventr001,belld001,zeilt001,ordor001,hampm001
1,20000330,0,Thu,NYN,NL,2,CHN,NL,2,5,...,reedr002,youne001,bufod001,sosas001,gracm001,rodrh001,andrs001,husoj001,giraj001,farnk001
2,20000403,0,Mon,COL,NL,1,ATL,NL,1,0,...,astap001,veraq001,sandr002,jonec004,jordb001,galaa001,jonea002,peree002,weisw001,maddg002
3,20000403,0,Mon,SFN,NL,1,FLO,NL,1,4,...,hernl003,castl001,gonza002,floyc001,wilsp002,lowem001,millk005,browb003,redmm001,ferna001
4,20000403,0,Mon,LAN,NL,1,MON,NL,1,10,...,browk001,bergp001,vidrj001,whitr001,guerv001,stevl001,barrm003,widgc001,cabro001,hermd001


In [222]:
# Combine the v_leage and h_league into one feature: intra_league 
games['intra_league'] = games.apply(lambda x: 1 if x['h_league']==x['v_league'] else 0, axis=1)
games.drop(['v_league','h_league'],axis=1, inplace=True)

In [226]:
# Make sure the game logs are sorted by date of game and reset the index
games = games.sort_values('date').reset_index(drop=True)

In [227]:
# How big is this games table?
print(games.shape)
#(43720, 96)

# When I loop through the game log table, I am going to update a column for "num_injuries" at any 
# game where we know a player on the disabled list played in THAT game and went on the DL right after.

# Create a column of zeros for num of injuries column to append to the games df that matches
# the games row length: 43720 rows long
num_injur_col = np.zeros((43720,), dtype=np.int)
# Add it to the games dataframe
games['injury']  = num_injur_col

(43720, 95)


In [166]:
# Create a small subset of the games dataframe to only include the game data and all the players, cols 70:93
games_day = games.iloc[:,0:2].copy()
sm_games = games.iloc[:,70:94].copy()
sm_games = pd.concat([games_day,sm_games],axis=1)

# FOR PRACTICE RUNS THROUGH THE FOR LOOPS:
# Subset dataframes to practice on:
sm_games = sm_games.iloc[:200,:]
sm_inj = injury.iloc[:150,:].to_frame().T.copy()

In [231]:
# Create a series to update in each for loop in the form of a dictionary:
injury_update_dict = games['injury'].to_dict()
sum(injury_update_dict.values())

0

In [240]:
# Define a function to search through the games and find out if player 'hamid001' played in the game
# We get the results which can be hundreds of rows of game data.
#  I want to find the last game hamid001 played before the 20000408 15-day DL hamid001 placement:

# Start with first day of 'dl' disabled list: 20000323
dl_day = 20000323
game_counter = 0
injury_found = 0
for dl_place in injury.itertuples():
    s_index = pd.Series()
    if dl_place[3] is not np.nan:
        pid = dl_place[3]
        dl_day = dl_place[1]
        last_play_day = dl_day
        for game in sm_games.itertuples():
            if pid in game:
                if game[1] < dl_day:
                    s_index.add(game[0])   
        if len(s_index) > 0:
            print(s_index.idxmax())
            index_needed = s_index.idxmax()
            injury_update_dict[index_needed] = 1
sum(injury_update_dict.values())

0

In [187]:
game_counter

8015

In [182]:
injury_dict

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 1,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 12

In [ ]:
# One Hot Encode before saving to CSV

In [ ]:
col = []
patsy.dmatrix('~ sale_type + paved_drive + garage_type +\
                    garage_finish + neighborhood + house_style + central_air +\
                    condition_1 + bldg_type -1', data = col,return_type = 'dataframe')

## Save File to CSV for Modeling 